# Script que crea las entidades NGSI-LD de Orion y sube los históricos a Timescale 

## Funciones empleadas para subir y modificar datos

In [1]:
!pip install pyarrow
import pickle


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pickle

### Creación de una rambla en Orion Context Broker

In [2]:
import requests
import json
import math

def crear_rambla(id, dateLastValueReported, description, locality, postalCode, source, location, 
                 section, nextSection, previousSection, name, isPartOf):
    rambla_content = {
        "id": f"urn:ngsi-ld:Ravine:{id}",
        "identifier":{
            "type": "Property",
            "value": f"Ravine:{id}"
        },
        "type": "Ravine",
        "description": {
            "type": "Property",
            "value": description
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode,
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }

    # Función auxiliar para agregar una propiedad si no es None
    def add_property_if_not_none(property_name, value):
            
        if isinstance(value, str):
            rambla_content[property_name] = {
                "type": "Relationship",
                "object": value
            }
        elif value is not None:
            rambla_content[property_name] = {
                "value": value
            }        

    # Agregar propiedades si no son None
    add_property_if_not_none("nextSection", nextSection)
    add_property_if_not_none("previousSection", previousSection)
    add_property_if_not_none("section", section)
    add_property_if_not_none("isPartOf", isPartOf)

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }
    
    response = requests.post(url, headers=headers, json=rambla_content)
    

### Creación de un Dispositivo en Orion Context Broker

In [3]:
def crear_device(id, alternateName, controlled_entity, controlled_properties, description, postalCode, locality, location, name, source, dateLastValueReported):
    device_content = {
        "id": f"urn:ngsi-ld:Device:{id}",
        "alternateName": {
            "type": "Property",
            "value": alternateName
        },
        "areaServed": {
            "type": "Property",
            "value": "Mar Menor"
        },
        "type": "Device",
        "controlledAsset" : {
            "type": "Relationship",
            "object": [
                controlled_entity
            ]
        },
        "controlledProperty": {
            "type": "Property",
            "value": controlled_properties
        },
        "dateLastValueReported": {
            "type": "Property",
            "value": dateLastValueReported
        },
        "description": {
            "type": "Property",
            "value": description
        },
        "deviceCategory": {
            "type": "Property",
            "value": [
                "sensor"
            ]
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }
    response = requests.post(url, headers=headers, json=device_content)
    print("Device")
    print(device_content)
    print(response.status_code)
    

### Creación de una Medición de un Dispositivo

In [4]:
def crear_device_measurement(id, deviceType, value, controlledProperty, observedAt, depth, target , refDevice, unitCode, name, source, dateLastValueReported):
    device_measure_content = {
        "id": f"urn:ngsi-ld:DeviceMeasurement:{id}",
        "deviceType": {
            "type": "Property",
            "value": deviceType
        },
        "type": "DeviceMeasurement",
        "controlledProperty": {
            "type": "Property",
            "value": controlledProperty
        },
        "dateLastValueReported": {
            "type": "Property",
            "value": dateLastValueReported
        },
        "numValue":{
            "type": "Property",
            "value": value,
            "observedAt" : observedAt,
        },
        "refDevice": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:Device:{refDevice}",
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }

    if unitCode is not None:
        device_measure_content["unit"] = {
            "type": "Property",
            "value": unitCode
        }
    
    if math.isnan(value):
        device_measure_content["numValue"]["value"] = -99

    if depth is not None:
        device_measure_content["depth"] = {
            "type": "Property",
            "value": depth
        }

    if target is not None:
        device_measure_content["target"] = {
            "type": "Property",
            "value": target
        }


    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }
    print("DeviceMeasurement")
    print(device_measure_content)
    
    response = requests.post(url, headers=headers, json=device_measure_content)
    print(response.status_code)


### Creación de una red acuatica, que estara compuesta por un complejo de rambas, como la del Albujón

In [5]:
def crear_water_network(id, isComposedOf, description, name, location = [[[37.7138,-1.1874], [37.7157, -0.8588], [37.7646, -1.1439], [37.7138,-1.1874]]]):
    water_network_content = {
        "id": f"urn:ngsi-ld:WaterNetwork:{id}",
        "type": "WaterNetwork",
        "isComposedOf": isComposedOf,
        "description": {
            "type": "Property",
            "value": description
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Polygon",
                "coordinates": location
            }
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia"
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }
    response = requests.post(url, headers=headers, json=water_network_content)
    print("WaterNetwork")
    print(response.status_code)

### Creación de una red piezometrica a la que asociaremos diversos sondeos

In [6]:
def crear_piezometric_net(id, description, name, source = "http://155.54.95.167/" , location = [[[37.6410,-0.7628], [37.7171, -0.8621], [37.8293,-0.7844], [37.8367, -0.8037], [37.7225, -0.9086], [37.6178, -0.7755], [37.6410,-0.7628] ]]):
    water_network_content = {
        "id": f"urn:ngsi-ld:PiezometricNet:{id}",
        "identifier": {
            "type": "Property",
            "value": f"urn:ngsi-ld:PiezometricNet:{id}"
        },
        "type": "PiezometricNet",
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia"
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "description": {
            "type": "Property",
            "value": description
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Polygon",
                "coordinates": location
            }
        },
        "name" : {
            "type": "Property",
            "value": name
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }
    response = requests.post(url, headers=headers, json=water_network_content)
    print("PiezometricNet")
    print(response.status_code)

### Creación de un Sondeo, el cual pertenecera a una red piezometrica

In [7]:
import math

def crear_sounding(id, locality, postalCode, location, description, source, dateLastValueReported, numberInNetwork, name, isPartOf):

    piezometer_content = {
        "id": f"urn:ngsi-ld:SoundingPlace:{id}",
        "identifier":{
            "type": "Property",
            "value": f"SoundingPlace:{id}"
        },
        "type": "SoundingPlace",
        "category": {
            "type": "Property",
            "value": "sensor"
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode
            },
            "verified": {
                "type": "Property",
                "value": True
            }
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "description": {
            "type": "Property",
            "value": description
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "numberInNetwork": {
            "type": "Property",
            "value": numberInNetwork
        },
         "name": {
            "type": "Property",
            "value": name
        },
        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }

    if isPartOf is not None:
        piezometer_content["isPartOf"] = {
            "type": "Relationship",
            "object": isPartOf
        }

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }


    response = requests.post(url, headers=headers, json=piezometer_content)
    print(piezometer_content)
    print(response)


### Creación de una Boya

In [8]:
import requests
import json

def crear_boya(id, dateLastValueReported, description, postalCode, locality, location, name, source, closeMeasurements=None):
    buoy_content = {
        "id": f"urn:ngsi-ld:Buoy:{id}",
        "identifier":{
            "type": "Property",
            "value":  f"Buoy:{id}"
        },
        "type": "Buoy",
        "description": {
            "type": "Property",
            "value": description
        },
        "address": {
            "type": "Property",
            "value": {
                "addressCountry": "ES",
                "addressRegion": "Murcia",
                "addressLocality": locality,
                "postalCode": postalCode
            },
        },
        "location": {
            "type": "GeoProperty",
            "value": {
                "type": "Point",
                "coordinates": location
            }
        },
        "name": {
            "type": "Property",
            "value": name
        },
        "source": {
            "type": "Property",
            "value": source
        },
        "closeMeasurements": closeMeasurements,

        "@context": "https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld"
    }
    
    if closeMeasurements is None:
        del buoy_content["closeMeasurements"]

    url = "http://localhost:1026/ngsi-ld/v1/entities/"

    headers = {
        "Content-Type": "application/ld+json"
    }

    try:
        response = requests.post(url, headers=headers, json=buoy_content)
        response.raise_for_status()  # Lanza una excepción para códigos de estado HTTP no exitosos
    except requests.exceptions.RequestException as e:
        print(f"Error creating entity: {e}")

### Modificación de una entidad, principalmente usado para el DeviceMeasurement con numValue



Tambien se emplea para modificar el isComposedOf del WaterNetwork

In [9]:
def modify_entity( value, observedAt, unitcode, url, depth = None, place = None, isComposedOf = False):

    headers = {
        'Content-Type': 'application/json',
        'Link': '<https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

    if isinstance(value, float) and  math.isnan(value):
        value = -99

    modify_content = {
        "numValue": {
            "type": "Property",
            "value": value,
            "observedAt": observedAt
        },
        "dateLastValueReported": {
            "type": "Property",
            "value": observedAt
        }
    }

    if depth is not None:
        modify_content["numValue"]["depth"] = {
            "type": "Property",
            "value": depth
        }
    
    if place is not None:
        modify_content["numValue"]["measurementPlace"] = {
            "type": "Property",
            "value": place
        }

    if unitcode is not None:
        modify_content["numValue"]["unitCode"] = unitcode

    if isComposedOf:
        modify_content = {
            "isComposedOf": value
        }


    print("modify_content")
    print(modify_content)
    response = requests.patch(url, headers=headers, json=modify_content)
    print(response.status_code)
    try:
        if response.status_code != 204:
            print(response.text)
    finally:
        response.close()

### Función empleada para añadir atributos a una entidad

In [10]:
def create_attribute(newAttributeName, value, observedAt, url, multipleValue = None):

    headers = {
        'Content-Type': 'application/json',
        'Link': '<https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

    if multipleValue is not None:
        modify_content = {
            newAttributeName: {
                 "value": [multipleValue]
            }
        }
    elif observedAt is None:
        modify_content = {
            newAttributeName: {
                "value": value
            }
        }
    else:       
        modify_content = {
            newAttributeName: {
                "value": value,
                "observedAt": observedAt
            }
        }

    response = requests.post(url, headers=headers, json=modify_content)
    try:
        if response.status_code != 204:
            print(response.text)
    finally:
        response.close()

### Función que realiza una petición para buscar las entidades cercanas a una entidad basado en sus coordenadas

In [11]:
def search_close_entities(coordinate_lat,coordinate_long, id_entity):
    url = 'http://localhost:1026/ngsi-ld/v1/entities'
    headers = {
        'Accept': 'application/json',
        'Link': '<https://raw.githubusercontent.com/yuye188/MarMenorDT/main/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

    params = {
        'geometry': 'Point',
        'coordinates': f'[{coordinate_lat},{coordinate_long}]',
        'georel': 'near;maxDistance==4000',
        'options': 'keyValues',
    }

    response = requests.get(url, headers=headers, params=params)

    data = response.json()

    filtered_data = [entity.get("id",) for entity in data if (id_entity not in entity.get('id', '') )]
    
    filtered_data = [ entity for entity in filtered_data if "Buoy" in entity or "SoundingPlace" in entity or "Ravine" in entity] 

    return filtered_data 
    

## Codigo para la subida de los datos

### Subida datos Boyas

Creamos un diccionario donde almacenaremos el id de las entidades y sus coordenadas para buscar posteriormente sus entidades cercanas y otras utilidades

In [12]:
entities_id = {}

#### Información estática de las Boyas que emplearemos para crearlas.

En caso de añadir nuevas ramblas habrá que añadirlas a este diccionario

In [13]:
diccionario_info_boyas = {
    "E1": {
        "locality": "Mar Menor, San Javier, Cartagena",
        "postalCode": "30380",
        "location": [37.81, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E1",
    },
    "E2": {
        "locality": "Mar Menor, Playa de la Hita, Cartagena",
        "postalCode": "30380",
        "location": [37.76, -0.8],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E2",
    },
    "E3": {
        "locality": "Mar Menor, Playa de la Hita, Cartagena",
        "postalCode": "30380",
        "location": [37.76, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E3",
    },
    "E4": {
        "locality": "Mar Menor, La Manga, Cartagena",
        "postalCode": "30380",
        "location": [37.74, -0.74],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E4",
    },
    "E5": {
        "locality": "Mar Menor, La Manga, Cartagena",
        "postalCode": "30380",
        "location": [37.74, -0.72],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E5",
    },
    "E6": {
        "locality": "Mar Menor, Isla Mayor, Cartagena",
        "postalCode": "30720",
        "location": [37.71, -0.77],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E6",
    },
    "E7": {
        "locality": "Mar Menor, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.71, -0.83],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E7",
    },
    "E8": {
        "locality": "Mar Menor, Los Urrutias, Cartagena",
        "postalCode": "30368",
        "location": [37.69, -0.81],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E8",
    },
    "E9": {
        "locality": "Mar Menor, Saladar de Lo Poyo, Cartagena",
        "postalCode": "30368",
        "location": [37.66, -0.8],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E9",
    },
    "E10": {
        "locality": "Mar Menor, Los Nietos, Cartagena",
        "postalCode": "30383",
        "location": [37.65, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E10",
    },
    "E11": {
        "locality": "Mar Menor, La Manga, Cartagena",
        "postalCode": "30380",
        "location": [37.65, -0.72],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E11",
    },
    "E12": {
        "locality": "Mar Menor, Cartagena",
        "postalCode": "30380",
        "location": [37.68, -0.78],
        "unitCode": "MTR",
        "source": "https://idearm.imida.es/cgi/siomctdmarmenor/#CTD-E12",
    },
}

#### Lectura de los .csv y unión de estos

In [14]:
import os
import glob
import pandas as pd
import time

# Especifica la ruta de la carpeta principal
carpeta = "/home/thinking/raw/BoyasProf/"

# Utiliza os.listdir() para obtener una lista de elementos en la carpeta
elementos_en_carpeta = os.listdir(carpeta)

# Filtra solo las carpetas en la lista de elementos
carpetas = [elemento for elemento in elementos_en_carpeta if os.path.isdir(os.path.join(carpeta, elemento))]

# DataFrame vacío para almacenar los datos
dfs = {}

# Imprime la lista de carpetas
for carpeta_actual in carpetas:
    print(carpeta_actual)
    df_total = pd.DataFrame()
    archivos_csv = glob.glob(os.path.join(carpeta+carpeta_actual, "*.csv"))
    for archivo in archivos_csv:
        
        nombre_archivo = archivo.split("/")[-1].split(".")[0]
        df_temporal = pd.read_csv(archivo)
        df_temporal.columns = [df_temporal.columns[0]] + [f"{nombre_archivo}_{col.split('_')[1]}" for col in df_temporal.columns[1:]]
        df_temporal = df_temporal.loc[:, ~df_temporal.columns.str.contains("None")]
        df_temporal.rename(columns={'fecha': 'Date'}, inplace=True)
        
        if(len(df_total) == 0):
            df_total = df_temporal
        else:
            #Hacer un merge de df_total y df_temporal por la columna "Date"
            df_total = pd.merge(df_total, df_temporal, on="Date", how="outer")
    dfs[carpeta_actual] = df_total

for key, df in dfs.items():
    # Convierte la columna "Date" a tipo datetime
    df["Date"] = pd.to_datetime(df["Date"])
    # Formatea la columna "Date" en el nuevo formato
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

CTD-E3
CTD-E12
CTD-E11
CTD-E10
CTD-E9
CTD-E4
CTD-E1
CTD-E7
CTD-E8
CTD-E6
CTD-E5
CTD-E2


#### Codigo de creación de las entidades actuales en el Orion Context

En este codigo se crean las entidades en el Orion Context Broker (OCB) y se realizan un par de modificaciones, las cuales al realizarse Mintaka se encargara de almacenar los cambios en el Timescale-DB (base de datos temporal). Simplemente realizaremos un par de modificaciones para que se creen los schemas en Timescale.

In [15]:
import math
import time
import re

# Diccionario que asocia las columnas del csv con el nombre del sensor que le asociemos
maper_property_sensor = { "Clorofila":"ChlorophyllSensor","PH":"PhSensor","Polietileno":"PolyethyleneSensor","Turbidez":"TurbiditySensor","Salinidad":"SalinitySensor",
        "Transparencia":"TransparencySensor","Oxigeno":"OxygenSensor","Temperatura": "TemperatureSensor", "Conductividad": "ConductivitySensor", "Piezometrico":"PiezometricLevelSensor", 
        "Total solido disuelto (TDS)": "TdsSensor", "Materia Organiza":"OrganicMatterSensor"
}

# Diccionario que asocia las columnas del csv con el nombre de la propiedad controlada que hayamos indicado en el modelo .yaml
maper_property_controlled = { "Temperatura": "temperature","Materia Organiza":"OrganicMatterSensor", "Conductividad": "conductivity", "Piezometrico":"piezometricLevel", "Salinidad":"salinity",
       "Total solido disuelto (TDS)": "tds", "Clorofila":"chlorophyll","PH":"pH","Polietileno":"polyethylene","Turbidez":"turbidity","Transparencia":"transparency","Oxigeno":"oxygen"
}

# Diccionario que asocia las columnas del csv con el nombre de la unidad de medida que le asociemos
maper_property_unitcode = { "Temperatura": "CEL", "Conductividad": "microseconds/cm", "Piezometrico":"msnm", "Salinidad":"PSU",
       "Total solido disuelto (TDS)": "mg/l", "Materia Organiza":None,"Clorofila":None,"PH":"pH",
       "Polietileno":None,"Turbidez":"NTU","Transparencia":None,"Oxigeno":None
}

# Diccionario que asocia el ID de un DeviceMeasurement con el ID de una entidad y 
# el nombre de la propiedad controlada que mide
# Con este diccionario podremos realizar posteriormente la modificación de las entidades
link_boya_sensores = {}

# Diccionario en el que se almacenarán las entidades de las boyas asociadas a cada carpeta
boya_entity = {}

# Contador que indica el numero de Boyas y Devices que se han creado
num_boyas = 0
num_devices = 0

# Contador que indica el numero de dispositivo de mediciones que se han creado
counter_device_measurements = 0

# Recorremos por cada Boya
for carpeta, valorCarpeta in dfs.items():
    print(carpeta)
    # Obtenemos los nombres de las columnas del csv
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    
    # Obtenemos el ID de la boya a partir del nombre de la carpeta 
    id_boya = carpeta.split("-")[1]
    # Creamos los ID de las entidades y dispositivo
    number_entity = str(num_boyas).zfill(3)
    number_device = str(num_devices).zfill(3)
    id_entity = f"urn:ngsi-ld:Buoy:{number_entity}"

    boya_entity[carpeta] = id_entity
    
    # Obtenemos las propiedades controladas por el dispositivo
    controlled_properties = [maper_property_controlled[valor] for valor in sensores if valor != "Date"]

    # Creamos el dispositivo
    crear_device(
        id=number_device,
        alternateName = "Device" + number_device + " for Buoy "+ number_entity,
        controlled_entity = id_entity,
        controlled_properties = controlled_properties,
        description = "Buoy "+ id_boya +" for measuring water quality in Mar Menor",
        postalCode = diccionario_info_boyas[id_boya]["postalCode"],
        locality = diccionario_info_boyas[id_boya]["locality"],
        location = diccionario_info_boyas[id_boya]["location"],
        name = "Buoy " + id_boya+" found in Mar Menor",
        source = diccionario_info_boyas[id_boya]["source"],
        dateLastValueReported=valorCarpeta["Date"].iloc[len(valorCarpeta)-1]
    )
    
    # Recorremos por cada fila del csv
    for index, row in valorCarpeta.iterrows():
        # Primera fila del csv
        if index == 0:
            # Recorremos por cada sensor
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue
                
                # Recorremos todas las columnas que empiecen por ese sensor ya que en los csv hay varias segun la profundidad
                for columnName in [ col for col in valorCarpeta.columns if col.startswith(valorSensor)]:
                    depth = columnName.split("_")[1]
                    depth = float(depth)

                    # Creamos el ID del device measurement y la entidad
                    id_device_measurement = f"urn:ngsi-ld:DeviceMeasurement:{str(counter_device_measurements).zfill(3)}"

                    crear_device_measurement(
                        id=str(counter_device_measurements).zfill(3),
                        deviceType=maper_property_sensor[sensores[contador]],
                        value=row[columnName],
                        controlledProperty=maper_property_controlled[valorSensor],
                        observedAt=row["Date"],
                        depth=depth,
                        target=None,
                        refDevice=number_device,
                        unitCode=maper_property_unitcode[valorSensor],
                        name="DeviceMeasurement "+ str(counter_device_measurements).zfill(3),
                        source=diccionario_info_boyas[id_boya]["source"],
                        dateLastValueReported=row["Date"]
                    )
                    counter_device_measurements = counter_device_measurements + 1
                    
                    # Asociamos el id del device measurement con el id de la entidad y el nombre de la propiedad controlada
                    if id_entity not in link_boya_sensores:
                        link_boya_sensores[id_entity] = {}
                    if valorSensor not in link_boya_sensores[id_entity]:
                        link_boya_sensores[id_entity][valorSensor] = []
                    link_boya_sensores[id_entity][valorSensor].append(id_device_measurement),
                
            # Creamos la boya
            entities_id[id_entity] = diccionario_info_boyas[id_boya]["location"]
            crear_boya(
                id=number_entity,
                dateLastValueReported=row["Date"],
                description="Buoy "+ id_boya +" for measuring water quality in the Mar Menor",
                postalCode=diccionario_info_boyas[id_boya]["postalCode"],
                location=diccionario_info_boyas[id_boya]["location"],
                locality=diccionario_info_boyas[id_boya]["locality"],
                name="Buoy " + id_boya,
                source=diccionario_info_boyas[id_boya]["source"]
            )
            num_boyas += 1
        
        # si el indice es 1 entonces haremos la modificación de las entidades
        elif index == 1:
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue
                
                # recorremos por cada propiedad y profundidad
                for number, columnName in enumerate([ col for col in valorCarpeta.columns if col.startswith(valorSensor)]):
                    depth = columnName.split("_")[1]
                    depth = float(depth)
                    
                    # recuperamos el id del device measurement a partir de la entidad y el nombre de la propiedad controlada
                    id_device_measurement = link_boya_sensores[id_entity][valorSensor][number]

                    # modificamos la entidad
                    modify_entity(
                        value=row[columnName],
                        observedAt=row["Date"],
                        unitcode=maper_property_unitcode[valorSensor],
                        url="http://localhost:1026/ngsi-ld/v1/entities/"+id_device_measurement+"/attrs",
                    )
            # 1 segundo para no saturar la red 
            time.sleep(1)
        else:
            break
    num_devices = num_devices+1

with open('./variables_entidades/boya_entity.pkl', 'wb') as f:
    pickle.dump(boya_entity, f)
with open('./variables_entidades/link_boya_sensores.pkl', 'wb') as f:
    pickle.dump(link_boya_sensores, f)

CTD-E3
Device
{'id': 'urn:ngsi-ld:Device:000', 'alternateName': {'type': 'Property', 'value': 'Device000 for Buoy 000'}, 'areaServed': {'type': 'Property', 'value': 'Mar Menor'}, 'type': 'Device', 'controlledAsset': {'type': 'Relationship', 'object': ['urn:ngsi-ld:Buoy:000']}, 'controlledProperty': {'type': 'Property', 'value': ['conductivity', 'transparency', 'pH', 'polyethylene', 'oxygen', 'temperature', 'turbidity', 'OrganicMatterSensor', 'chlorophyll', 'salinity']}, 'dateLastValueReported': {'type': 'Property', 'value': '2023-08-24T00:00:00Z'}, 'description': {'type': 'Property', 'value': 'Buoy E3 for measuring water quality in Mar Menor'}, 'deviceCategory': {'type': 'Property', 'value': ['sensor']}, 'address': {'type': 'Property', 'value': {'addressCountry': 'ES', 'addressRegion': 'Murcia', 'addressLocality': 'Mar Menor, Playa de la Hita, Cartagena', 'postalCode': '30380'}, 'verified': {'type': 'Property', 'value': True}}, 'source': {'type': 'Property', 'value': 'https://idearm.im

#### Conexión directa con Timescale para subir los datos sin pasar por ORION

Aqui subiremos los datos históricos sin pasar por ORION, ya que ORION solo le importan los datos actuales y asi se agiliza enormemente el proceso

In [16]:
import psycopg2
from datetime import datetime
import uuid
# Conectarse a la base de datos
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="orion",
    user="orion",
    password="orion"
)

cursor = conn.cursor()
datos_a_insertar = []

# Recorremos las distintas Boyas
for carpeta, valorCarpeta in dfs.items():
    # Obtenemos el ID de la boya a partir del nombre de la carpeta
    id_entity = boya_entity[carpeta]
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    # Recorremos por cada fila del csv
    for index, row in valorCarpeta.iterrows():
        if index == len(valorCarpeta)-1 or index == 0 or index == 1:
            continue
        # Para cada columna del csv
        for contador,valorSensor in enumerate(sensores):
            if valorSensor == "Date":
                continue
            
            # Recorremos todas las columnas que empiecen por ese sensor ya que en los csv hay varias segun la profundidad
            for number, columnName in enumerate([ col for col in valorCarpeta.columns if col.startswith(valorSensor)]):
                depth = columnName.split("_")[1]
                depth = float(depth)
                
                # Recuperamos el ID del device measurement a partir de la entidad y el nombre de la propiedad controlada
                id_device_measurement = link_boya_sensores[id_entity][valorSensor][number]

                fecha_hora = datetime.strptime(row["Date"], "%Y-%m-%dT%H:%M:%SZ")
                
                value = row[columnName]
                if math.isnan(row[columnName]):
                    value = -99

                # Almacenamos en una variable los datos que vamos a insertar habiendonos fijado en el esquema de la base de datos
                datos_a_insertar.append(
                    (f'urn:ngsi-ld:attribute:{uuid.uuid4()}', 'https://smartdatamodels.org/dataModel.DeviceMeasurement/numValue', 'Replace', id_device_measurement, fecha_hora, True, maper_property_unitcode[valorSensor], 'None', 'Number', None, None, value, None, None, None, None, None, None, None, None, datetime.now())
                )
                
                # En vez de insertar los datos en la base de datos cada vez que se recorre una fila del csv,
                # se almacenan en una lista y se insertan en la base de datos cuando la lista tiene más de 100000 filas
                if len(datos_a_insertar) > 100000:
                    consulta_insercion = f"INSERT INTO attributes VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                    print(f"Insertando {len(datos_a_insertar)} filas en la tabla attributes")
                    try:
                        # Ejecutar la consulta para insertar los datos
                        cursor.executemany(consulta_insercion, datos_a_insertar)
                        conn.commit()
                        datos_a_insertar = []
                        print(f"Se han insertado {cursor.rowcount} filas en la tabla attributes")
                    except (Exception, psycopg2.DatabaseError) as error:
                        conn.rollback()
                    # Confirmar la transacción
                        print(f"Error: {error}")

# Insertar los datos restantes en la base de datos
if len(datos_a_insertar) > 0:
    consulta_insercion = f"INSERT INTO attributes VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    print(f"Insertando {len(datos_a_insertar)} filas en la tabla attributes")
    try:
        # Ejecutar la consulta para insertar los datos
        cursor.executemany(consulta_insercion, datos_a_insertar)
        conn.commit()
        datos_a_insertar = []
        print(f"Se han insertado {cursor.rowcount} filas en la tabla attributes")
    except (Exception, psycopg2.DatabaseError) as error:
        conn.rollback()
    # Confirmar la transacción
        print(f"Error: {error}")

                

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 22516 filas en la tabla attributes
Se han insertado 22516 filas en la tabla attributes


#### Queda actualizar las entidades de ORION con los ultimos valores

In [17]:
for carpeta, valorCarpeta in dfs.items():
    # Obtenemos los ultimos valores de cada columna del csv
    ultimos_valores = valorCarpeta.iloc[len(valorCarpeta)-1]
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    id_entity = boya_entity[carpeta]
    # Recorremos por cada columna del csv
    for contador,valorSensor in enumerate(sensores):
        if valorSensor == "Date":
            continue

        for number, columnName in enumerate([ col for col in valorCarpeta.columns if col.startswith(valorSensor)]):
            depth = columnName.split("_")[1]
            depth = float(depth)

            id_device_measurement = link_boya_sensores[id_entity][valorSensor][number]

            modify_entity(
                value=ultimos_valores[columnName],
                observedAt=ultimos_valores["Date"],
                unitcode=maper_property_unitcode[valorSensor],
                url="http://localhost:1026/ngsi-ld/v1/entities/"+id_device_measurement+"/attrs",
            )

modify_content
{'numValue': {'type': 'Property', 'value': 7.13, 'observedAt': '2023-08-24T00:00:00Z', 'unitCode': 'microseconds/cm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2023-08-24T00:00:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 7.13, 'observedAt': '2023-08-24T00:00:00Z', 'unitCode': 'microseconds/cm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2023-08-24T00:00:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 7.17, 'observedAt': '2023-08-24T00:00:00Z', 'unitCode': 'microseconds/cm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2023-08-24T00:00:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 7.26, 'observedAt': '2023-08-24T00:00:00Z', 'unitCode': 'microseconds/cm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2023-08-24T00:00:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 7.26, 'observedAt': '2023-08-24T00:00:00Z', 'unitCode': 'microseconds/

#### Codigo para obtener información del esquema y poder ver ejemplos 

In [18]:
import psycopg2
import datetime
# Conectarse a la base de datos
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="orion",
    user="orion",
    password="orion"
)
cursor = conn.cursor()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")

# Iterar sobre las tablas
for tabla in cursor.fetchall():

    nombre_tabla = tabla[0]
    print(nombre_tabla)
    if nombre_tabla == "subattributes":
        print(f"Esquema de la tabla '{nombre_tabla}':")
        
        # Obtener información sobre las columnas de la tabla
        cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{nombre_tabla}'")
        
        # Mostrar el esquema de la tabla
        for columna in cursor.fetchall():
            print(columna)

        cursor.execute(f"SELECT * FROM {nombre_tabla}")
        contador = 0
        # Mostrar el esquema de la tabla
        for columna in cursor.fetchall():
            contador += 1
            print(columna)
            if contador > 1500000:
                break
        #('urn:ngsi-ld:attribute:instance:df60ec4c-d714-11ee-aec1-0242ac120105', 'https://smartdatamodels.org/depth', 'urn:ngsi-ld:DeviceMeasurement:006', 'urn:ngsi-ld:attribute:instance:df60ec2e-d714-11ee-aec1-0242ac120105', 'None', None, None, 'String', '-1.6', None, None, None, None, None, None, None, None, None, None, datetime(2024, 2, 29, 15, 11, 54, 204000))


# Cerrar el cursor y la conexión
cursor.close()
conn.close()

spatial_ref_sys
geography_columns
geometry_columns
raster_columns
raster_overviews
entities
attributes
subattributes
Esquema de la tabla 'subattributes':


('instanceid', 'text')
('id', 'text')
('entityid', 'text')
('attrinstanceid', 'text')
('attrdatasetid', 'character varying')
('observedat', 'timestamp without time zone')
('unitcode', 'text')
('valuetype', 'USER-DEFINED')
('text', 'text')
('boolean', 'boolean')
('number', 'double precision')
('datetime', 'timestamp without time zone')
('compound', 'jsonb')
('geopoint', 'USER-DEFINED')
('geomultipoint', 'USER-DEFINED')
('geopolygon', 'USER-DEFINED')
('geomultipolygon', 'USER-DEFINED')
('geolinestring', 'USER-DEFINED')
('geomultilinestring', 'USER-DEFINED')
('ts', 'timestamp without time zone')
('urn:ngsi-ld:attribute:instance:5bab90a2-2839-11ef-85f7-0242ac120104', 'https://uri.etsi.org/ngsi-ld/default-context/verified', 'urn:ngsi-ld:Device:000', 'urn:ngsi-ld:attribute:instance:5bab903e-2839-11ef-85f7-0242ac120104', 'None', None, None, 'Boolean', None, True, None, None, None, None, None, None, None, None, None, datetime.datetime(2024, 6, 11, 21, 27, 8, 341000))
('urn:ngsi-ld:attribute:in

#### Al haber tantos datos algunas consultas eran lentas por lo que se han creado Indices sobre los atributos mas empleados

In [19]:
import psycopg2
import datetime
# Conectarse a la base de datos
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="orion",
    user="orion",
    password="orion"
)

cur = conn.cursor()

# Crear índice en la columna 'datasetid'
cur.execute("CREATE INDEX idx_datasetid ON attributes (datasetid);")

# Crear índice en la columna 'entityid'
cur.execute("CREATE INDEX idx_entityid ON attributes (entityid);")

# Crear índice en la columna 'observedat'
cur.execute("CREATE INDEX idx_observedat ON attributes (observedat);")

# Confirmar los cambios
conn.commit()


# Cerrar el cursor y la conexión
cur.close()
conn.close()

### Subimos informacion de los Piezometros

#### Lectura csv/parquet y los juntamos 

In [20]:
import os
import glob
import pandas as pd
import pyarrow.parquet as pq


# Especifica la ruta de la carpeta principal
carpeta = "/home/thinking/raw/SAIHdatasActualizados/piezometros/"


# Utiliza os.listdir() para obtener una lista de elementos en la carpeta
elementos_en_carpeta = os.listdir(carpeta)

# Filtra solo las carpetas en la lista de elementos
carpetas = [elemento for elemento in elementos_en_carpeta if os.path.isdir(os.path.join(carpeta, elemento))]

# DataFrame vacío para almacenar los datos
dfs = {}


# Imprime la lista de carpetas
for carpeta_actual in carpetas:
    df_total = pd.DataFrame()
    archivos_parquet = glob.glob(os.path.join(carpeta+carpeta_actual, "*.parquet"))
    #archivos_csv = glob.glob(os.path.join(carpeta+carpeta_actual, "*.csv"))
    for archivo in archivos_parquet:
    
        #df_temporal = pd.read_csv(archivo)
        df_temporal = pd.read_parquet(archivo)
        df_temporal.to_csv(archivo.split(".")[0]+"from_parquet.csv", index=False)
        tabla = pq.ParquetFile(archivo)
        #pq.write_table(tabla, "ejemplo.parquet", version='1.0')
        # Imprime la versión del archivo Parquet
        print(tabla.metadata)
        print("-"*10)
        if(len(df_total) == 0):
            df_total = df_temporal
        else:
            #Hacer un merge de df_total y df_temporal por la columna "Date"
            df_total = pd.merge(df_total, df_temporal, on="Date", how="outer")
    dfs[carpeta_actual] = df_total

for key, df in dfs.items():
    # Convierte la columna "Date" a tipo datetime
    df["Date"] = pd.to_datetime(df["Date"])
    # Formatea la columna "Date" en el nuevo formato
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 2
  num_rows: 81492
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1685
----------
  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 2
  num_rows: 79476
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1565
----------
  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 2
  num_rows: 79476
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1561
----------
  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 2
  num_rows: 79476
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1537
----------
  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 2
  num_rows: 79476
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1557
----------
  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 2
  num_rows: 79476
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 1560
----------
  created_by: parquet-cpp-arrow version 10.0.1

#### Información estatica de los piezometros 

In [21]:
diccionario_info_piezometros = {
    "06Z01": {
        "locality": "Calle de Murillo, Los Belones, Cartagena",
        "postalCode": "30385",
        "location": [37.630082, -0.777017],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z01E10",
    },
    "06Z02": {
        "locality": "F-34, Cartagena",
        "postalCode": "30383",
        "location": [37.642663, -0.777275],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z02E10",
    },
    "06Z03": {
        "locality": "Camino Rural XVIII, El Algar, Cartagena",
        "postalCode": "30366",
        "location": [37.654261, -0.848734],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z03E10",
    },
    "06Z05": {
        "locality": "N-332, Cartagena",
        "postalCode": "30368",
        "location": [37.6841, -0.8692],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z05E10",
    },
    "06Z06": {
        "locality": "N-332, Cartagena",
        "postalCode": "30368",
        "location": [ 37.7058, -0.8633],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z06E10",
    },
    "06Z07": {
        "locality": "Camino Rural XIII, El Algar, Cartagena",
        "postalCode": "30395",
        "location": [37.7104, -0.8879],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z07E10",
    },
    "06Z08": {
        "locality": "F-35, Cartagena",
        "postalCode": "30710",
        "location": [37.7511, -0.8828],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z08E10",
    },
    "06Z09": {
        "locality": "Autopista del Mediterraneo, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.7452, -0.8679],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z09E10",
    },
    "06Z10": {
        "locality": "Cartagena",
        "postalCode": "30710",
        "location": [37.7618, -0.8673],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z10E10",
    },
    "06Z11": {
        "locality": "Avenida de Muñoz Zambudio, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.7543, -0.8588],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z11E10",
    },
    "06Z12": {
        "locality": "Calle Bergantin, San Javier, Cartagena",
        "postalCode": "30739",
        "location": [37.7755, -0.8514],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z12E10",
    },
    "06Z13": {
        "locality": "Calle de Poseidon, Los Alcazares, Cartagena",
        "postalCode": "30710",
        "location": [37.7696, -0.8385],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z13E10",
    },
    "06Z14": {
        "locality": "N-332, San Javier, Cartagena",
        "postalCode": "30730",
        "location": [37.7883, -0.8387],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z14E10",
    },
    "06Z15": {
        "locality": "Cartagena",
        "postalCode": "30730",
        "location": [37.7802, -0.8302],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z15E10",
    },
    "06Z16": {
        "locality": "Calle del Embalse de la Toba, San Javier, Cartagena",
        "postalCode": "30730",
        "location": [37.8181, -0.8286],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z16E10",
    },
    "06Z17": {
        "locality": "Avenida de la Romeria de San Blas 2, San Javier, Cartagena",
        "postalCode": "30720",
        "location": [37.8059, -0.8168],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z17E10",
    },
    "06Z18": {
        "locality": "Autopista del Mediterraneo, San Javier, Cartagena",
        "postalCode": "30730",
        "location": [37.8247, -0.8165],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z18E10",
    },
    "06Z19": {
        "locality": "Calle de Navacerrada 33, San Javier, Cartagena",
        "postalCode": "30720",
        "location": [37.8178, -0.8085],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z19E10",
    },
    "06Z20": {
        "locality": "Calle Francesco Borromini 3, San Pedro del Pinatar, Cartagena",
        "postalCode": "30740",
        "location": [37.8350, -0.8023],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06Z20E10",
    },
}

#### Subimos la red piezometrica

Esta Red incorporará los sondeos que se agrupan alrededor del mar menor

In [22]:
id_piezometric_net = "urn:ngsi-ld:PiezometricNet:000"

crear_piezometric_net(
    id = str(0).zfill(3), 
    description = "Piezometric Network of the Mar Menor, composed of several boreholes in the coastal area.", 
    name = "Piezometric Network of the Mar Menor",
)

PiezometricNet
201


#### Creamos los piezometros y dispositivos en el OCB

In [23]:
import math
import time

def check_exists(place_to_search, value_to_search, array=False):
    if array:
        return place_to_search[value_to_search][0] if value_to_search in place_to_search else None

    return place_to_search[value_to_search] if value_to_search in place_to_search else None



# Diccionario que asocia el nombre de la carpeta a la propiedad (CREO QUE NO SE USA)
maper_property_filename = { "Temperatura": "E01-Temperaturafrom_parquet.csv", "Conductividad": "E03-Conductividadfrom_parquet.csv", "Piezometrico":"E10-Piezometricofrom_parquet.csv", "Salinidad":"E11-Salinidadfrom_parquet.csv",
        "Total solido disuelto (TDS)": "E12-Total solido disuelto (TDS)from_parquet.csv",}

# Diccionarios comentados en el apartado de Boyas
maper_property_sensor = { "Temperatura": "TemperatureSensor", "Conductividad": "ConductivitySensor", "Piezometrico":"PiezometricLevelSensor", "Salinidad":"SalinitySensor",
       "Total solido disuelto (TDS)": "TdsSensor", "Solidez":"HardnesSensor"
}

maper_property_controlled = { "Temperatura": "temperature", "Conductividad": "conductivity", "Piezometrico":"piezometricLevel", "Salinidad":"salinity",
       "Total solido disuelto (TDS)": "tds"
}

maper_property_unitcode = { "Temperatura": "CEL", "Conductividad": "microseconds/cm", "Piezometrico":"msnm", "Salinidad":"PSU",
       "Total solido disuelto (TDS)": "mg/l"
}

link_piezometro_sensores = {}
num_piezometros = 0
piezometro_entity = {}

# Recorremos los distintos piezometros
for carpeta, valorCarpeta in dfs.items():
    print(carpeta)
    sensores = valorCarpeta.columns
    # Creamos IDs para las entidades y dispositivos
    id_entity = f"urn:ngsi-ld:SoundingPlace:{str(num_piezometros).zfill(3)}"
    number_entity = str(num_piezometros).zfill(3)
    number_device = str(num_devices).zfill(3)
    id_piezometro = carpeta.split("-")[0]

    piezometro_entity[carpeta] = id_entity
    controlled_properties = [maper_property_controlled[valor] for valor in sensores if valor != "Date"]

    crear_device(
        id=number_device,
        alternateName = "Multiple sensors for Sounding Place "+ id_piezometro,
        controlled_entity = id_entity,
        controlled_properties = controlled_properties,
        description = "Device from Piezometric Net, belonging to the Sounding Place "+ id_piezometro,
        postalCode = diccionario_info_piezometros[id_piezometro]["postalCode"],
        locality = diccionario_info_piezometros[id_piezometro]["locality"],
        location = diccionario_info_piezometros[id_piezometro]["location"],
        name = "Device " + number_device+" found in Sounding Place "+ id_piezometro,
        source = diccionario_info_piezometros[id_piezometro]["source"],
        dateLastValueReported=valorCarpeta["Date"].iloc[len(valorCarpeta)-1]
    )

    # Para cada fila del csv
    for index, row in valorCarpeta.iterrows():
        if index == 0:
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue

                # Creamos Id para el dispositivo de medicion y el dispositivo de medicion    
                id_device_measurement = f"urn:ngsi-ld:DeviceMeasurement:{str(counter_device_measurements).zfill(3)}"
                
                crear_device_measurement(
                    id=str(counter_device_measurements).zfill(3),
                    deviceType=maper_property_sensor[sensores[contador]],
                    value=row[valorSensor],
                    controlledProperty=maper_property_controlled[valorSensor],
                    observedAt=row["Date"],
                    depth=None,
                    target=None,
                    refDevice=number_device,
                    unitCode=maper_property_unitcode[valorSensor],
                    name="DeviceMeasurement "+ str(counter_device_measurements).zfill(3),
                    source=diccionario_info_piezometros[id_piezometro]["source"],
                    dateLastValueReported=row["Date"]
                )

                counter_device_measurements = counter_device_measurements + 1

                # Asociamos el id del dispositivo de medicion con el id de la entidad y el nombre de la propiedad controlada
                if id_entity not in link_piezometro_sensores:
                    link_piezometro_sensores[id_entity] = {}
                link_piezometro_sensores[id_entity][valorSensor] = id_device_measurement

            entities_id[id_entity] = diccionario_info_piezometros[id_piezometro]["location"]
            # Creamos el sondeo
            crear_sounding(
                id=number_entity,
                locality=diccionario_info_piezometros[id_piezometro]["locality"],
                postalCode=diccionario_info_piezometros[id_piezometro]["postalCode"],
                location=diccionario_info_piezometros[id_piezometro]["location"],
                description="Sounding Place "+ id_piezometro +" which belong to the Piezometric Network in Mar Menor",
                source=diccionario_info_piezometros[id_piezometro]["source"],
                dateLastValueReported=row["Date"],
                numberInNetwork=id_piezometro.split("Z")[1],
                name="Sounding Place "+id_piezometro,
                isPartOf= id_piezometric_net
            )
            num_piezometros += 1

        # Si el indice es 1 entonces haremos la modificación de las entidades
        elif index == 1:
            # Para cada columna del csv
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue
                # Recuperamos id del device y modificamos la entidad
                id_device_measurement = link_piezometro_sensores[id_entity][valorSensor]

                modify_entity(
                    value=row[valorSensor],
                    observedAt=row["Date"],
                    unitcode=maper_property_unitcode[valorSensor],
                    url="http://localhost:1026/ngsi-ld/v1/entities/"+id_device_measurement+"/attrs",
                )
                time.sleep(0.2)
        else:
            break
    num_devices = num_devices+1
with open('./variables_entidades/piezometro_entity.pkl', 'wb') as f:
    pickle.dump(piezometro_entity, f)
with open('./variables_entidades/link_piezometro_sensores.pkl', 'wb') as f:
    pickle.dump(link_piezometro_sensores, f)

06Z06-Sondeo06
Device
{'id': 'urn:ngsi-ld:Device:012', 'alternateName': {'type': 'Property', 'value': 'Multiple sensors for Sounding Place 06Z06'}, 'areaServed': {'type': 'Property', 'value': 'Mar Menor'}, 'type': 'Device', 'controlledAsset': {'type': 'Relationship', 'object': ['urn:ngsi-ld:SoundingPlace:000']}, 'controlledProperty': {'type': 'Property', 'value': ['tds', 'conductivity', 'piezometricLevel', 'salinity', 'temperature']}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}, 'description': {'type': 'Property', 'value': 'Device from Piezometric Net, belonging to the Sounding Place 06Z06'}, 'deviceCategory': {'type': 'Property', 'value': ['sensor']}, 'address': {'type': 'Property', 'value': {'addressCountry': 'ES', 'addressRegion': 'Murcia', 'addressLocality': 'N-332, Cartagena', 'postalCode': '30368'}, 'verified': {'type': 'Property', 'value': True}}, 'source': {'type': 'Property', 'value': 'https://saihweb.chsegura.es/apps/iVisor/visor_variable.ph

#### Conexion con TimeScale y insertamos los datos como con las Boyas

In [24]:
import psycopg2
from datetime import datetime
import uuid
# Conectarse a la base de datos
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="orion",
    user="orion",
    password="orion"
)

cursor = conn.cursor()
datos_a_insertar = []

for carpeta, valorCarpeta in dfs.items():
    id_entity = piezometro_entity[carpeta]
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    for index, row in valorCarpeta.iterrows():
        if index == len(valorCarpeta)-1 or index == 0 or index == 1:
            continue
        for contador,valorSensor in enumerate(sensores):
            if valorSensor == "Date":
                continue


            id_device_measurement = link_piezometro_sensores[id_entity][valorSensor]

            fecha_hora = datetime.strptime(row["Date"], "%Y-%m-%dT%H:%M:%SZ")

            value = row[valorSensor]
            if math.isnan(row[valorSensor]):
                    value = -99

            datos_a_insertar.append(
                (f'urn:ngsi-ld:attributes2:{uuid.uuid4()}', 'https://smartdatamodels.org/dataModel.DeviceMeasurement/numValue', 'Replace', id_device_measurement, fecha_hora, True, maper_property_unitcode[valorSensor], 'None', 'Number', None, None, value, None, None, None, None, None, None, None, None, datetime.now())
            )
            
            if len(datos_a_insertar) > 100000:
                consulta_insercion = f"INSERT INTO attributes VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                print(f"Insertando {len(datos_a_insertar)} filas en la tabla attributes")
                try:
                    # Ejecutar la consulta para insertar los datos
                    cursor.executemany(consulta_insercion, datos_a_insertar)
                    conn.commit()
                    datos_a_insertar = []
                    print(f"Se han insertado {cursor.rowcount} filas en la tabla attributes")
                except (Exception, psycopg2.DatabaseError) as error:
                    conn.rollback()
                # Confirmar la transacción
                    print(f"Error: {error}")

if len(datos_a_insertar) > 0:
    consulta_insercion = f"INSERT INTO attributes VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    print(f"Insertando {len(datos_a_insertar)} filas en la tabla attributes")
    try:
        # Ejecutar la consulta para insertar los datos
        cursor.executemany(consulta_insercion, datos_a_insertar)
        conn.commit()
        datos_a_insertar = []
        print(f"Se han insertado {cursor.rowcount} filas en la tabla attributes")
    except (Exception, psycopg2.DatabaseError) as error:
        conn.rollback()
    # Confirmar la transacción
        print(f"Error: {error}")

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes


#### Modificación de las entidades de ORION para que tengan los ultimos valores

In [25]:
for carpeta, valorCarpeta in dfs.items():
    ultimos_valores = valorCarpeta.iloc[len(valorCarpeta)-1]
    print(carpeta)
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    id_entity = piezometro_entity[carpeta]
    for contador,valorSensor in enumerate(sensores):
        if valorSensor == "Date":
            continue
        
        id_device_measurement = link_piezometro_sensores[id_entity][valorSensor]

        modify_entity(
            value=ultimos_valores[valorSensor],
            observedAt=ultimos_valores["Date"],
            unitcode=maper_property_unitcode[valorSensor],
            url="http://localhost:1026/ngsi-ld/v1/entities/"+id_device_measurement+"/attrs",
        )

06Z06-Sondeo06
modify_content
{'numValue': {'type': 'Property', 'value': -99, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'microseconds/cm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': -99, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'msnm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': -99, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'CEL'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': -99, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'PSU'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 9640.0, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'mg/l'}, 'dateLastValueReported': {'

#### Consulta para ver los esquemas 

In [26]:
''' Da un error que causa el crash del kernel
import psycopg2
import datetime
# Conectarse a la base de datos
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="orion",
    user="orion",
    password="orion"
)
cursor = conn.cursor()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")

# Iterar sobre las tablas
for tabla in cursor.fetchall():

    nombre_tabla = tabla[0]
    if nombre_tabla == "attributes":
        print(f"Esquema de la tabla '{nombre_tabla}':")
        
        # Obtener información sobre las columnas de la tabla
        cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{nombre_tabla}'")
        
        # Mostrar el esquema de la tabla
        for columna in cursor.fetchall():
            print(columna)

        cursor.execute(f"SELECT * FROM {nombre_tabla}")
        contador = 0
        # Mostrar el esquema de la tabla
        for columna in cursor.fetchall():
            contador += 1
            print(columna)
            if contador > 50000:
                break
        #('urn:ngsi-ld:attribute:instance:df60ec4c-d714-11ee-aec1-0242ac120105', 'https://smartdatamodels.org/depth', 'urn:ngsi-ld:DeviceMeasurement:006', 'urn:ngsi-ld:attribute:instance:df60ec2e-d714-11ee-aec1-0242ac120105', 'None', None, None, 'String', '-1.6', None, None, None, None, None, None, None, None, None, None, datetime(2024, 2, 29, 15, 11, 54, 204000))


# Cerrar el cursor y la conexión
cursor.close()
conn.close()
'''

' Da un error que causa el crash del kernel\nimport psycopg2\nimport datetime\n# Conectarse a la base de datos\nconn = psycopg2.connect(\n    host="localhost",\n    port="5432",\n    database="orion",\n    user="orion",\n    password="orion"\n)\ncursor = conn.cursor()\n\ncursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema=\'public\'")\n\n# Iterar sobre las tablas\nfor tabla in cursor.fetchall():\n\n    nombre_tabla = tabla[0]\n    if nombre_tabla == "attributes":\n        print(f"Esquema de la tabla \'{nombre_tabla}\':")\n        \n        # Obtener información sobre las columnas de la tabla\n        cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = \'{nombre_tabla}\'")\n        \n        # Mostrar el esquema de la tabla\n        for columna in cursor.fetchall():\n            print(columna)\n\n        cursor.execute(f"SELECT * FROM {nombre_tabla}")\n        contador = 0\n        # Mostrar el esquema de la t

### Subiendo la información de las Ramblas

#### Información estatica de las ramblas

In [27]:
# Se han establecido manualmente los ids, aunque no haría falta, se podría hacer como en las otras
maper_rambla_id = {
    "06A01": "urn:ngsi-ld:Ravine:001",
    "06A02": "urn:ngsi-ld:Ravine:002",
    "06A03": "urn:ngsi-ld:Ravine:003",
    "06A04": "urn:ngsi-ld:Ravine:004",
    "06A05": "urn:ngsi-ld:Ravine:005",
    "06A06": "urn:ngsi-ld:Ravine:006",
    "06A18": "urn:ngsi-ld:Ravine:007",
    "06A19": "urn:ngsi-ld:Ravine:008",
    "06P02": "urn:ngsi-ld:Ravine:009",
    "06P04": "urn:ngsi-ld:Ravine:010",
    "06P05": "urn:ngsi-ld:Ravine:011",
    "01M02": "urn:ngsi-ld:Ravine:012",
    "06B01": "urn:ngsi-ld:Ravine:013"
}

diccionario_info_ramblas = {
    "06A01": {
        "description": "Control frame 06A01 in La Puebla, Rambla del Albujon.",
        "locality": "RM-F35, La Puebla, Cartagena",
        "postalCode": "30395",
        "location": [37.7207, -0.9144],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A01U12",
        "section": 5,
        "nextSection": [maper_rambla_id["06A18"]],
        "previousSection": [maper_rambla_id["06A02"]],
    },
    "06A02": {
        "description": "Control frame 06A02 in Pozo Estrecho, Albujón watercourse.",
        "locality": "Pozo Estrecho, Cartagena",
        "postalCode": "30594",
        "location": [37.7239, -0.9793],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A02U12",
        "section": 4,
        "nextSection": [maper_rambla_id["06A01"]],
        "previousSection": [maper_rambla_id["06A03"]],
    },
    "06A03": {
        "description": "Control frame 06A03 Rambla del Albujón",
        "locality": "Rambla de El Albujón, Cartagena",
        "postalCode": "30330",
        "location": [37.7213, -1.0491],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A03U12",
        "section": 3,
        "nextSection": [maper_rambla_id["06A02"]],
        "previousSection": [maper_rambla_id["06A04"]],
    },
    "06A04": {
        "description": "Control frame 06A04 in el Estrecho, rambla del Albujón",
        "locality": "Estrecho de Fuente Álamo, Cartagena",
        "postalCode": "30332",
        "location": [37.7259, -1.1098],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A04U12",
        "section": 2,
        "nextSection": [maper_rambla_id["06A03"]],
        "previousSection": [maper_rambla_id["06A05"],maper_rambla_id["06A06"]],
    },
    "06A05": {
        "description": "Control frame 06A05 in Fuente Álamo, rambla del Albujón",
        "locality": "Fuente Álamo, Murcia",
        "postalCode": "30320",
        "location": [37.7247, -1.1721],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A05U12",
        "section": 1,
        "nextSection": [maper_rambla_id["06A04"]],
        "previousSection": None,
    },
    "06A06": {
        "description": "Control frame 06A06 in Los Cegarras, rambla de la Murta",
        "locality": "Cabecico del Rey, Murcia",
        "postalCode": "30154",
        "location": [37.7541, -1.1443],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A06U12",
        "section": 1,
        "nextSection": [maper_rambla_id["06A04"]],
        "previousSection": None,
    },
    "06A18": {
        "description": "Mouth 06A05 Rambla del Albujón",
        "locality": "Bahía Bella, Cartagena",
        "postalCode": "30368",
        "location": [37.7163, -0.8610],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A18U12",
        "section": 6,
        "nextSection": None,
        "previousSection": [maper_rambla_id["06A01"]],
    },
    "06A19": {
        "description": "Control frame in Rambla de la Maraña- Balsicas",
        "locality": "Los Camachos, Cartagena",
        "postalCode": "30592",
        "location": [37.7953, -0.9415],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A19U12",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06P02": {
        "description": "Control frame 06P02 in Rambla de la Maraña- Balsicas",
        "locality": "Torre Pacheco, Murcia",
        "postalCode": "30708",
        "location": [37.7492, -0.9921],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06P02P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06P04": {
        "description": "Control frame 06P04 in Torre Pacheco",
        "locality": "La Murta-Carrascoy, Murcia",
        "postalCode": "30153",
        "location": [37.8270, -1.2107],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06P04P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06P05": {
        "description": "Control frame 06P04 in San Javier (El Mirador)",
        "locality": "El Mirador, San Javier, Murcia",
        "postalCode": "30592",
        "location": [37.8359, -0.8757],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06P05P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "01M02": {
        "description": "Meteorological-Hydrological Station 01M02 el Relojero (AMETSE)",
        "locality": "El Relojero, La Alberca, Murcia",
        "postalCode": "30155",
        "location": [37.9136, -1.1190],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=01M02P01",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    },
    "06B01": {
        "description": "Elevation 06B01 el Albujón",
        "locality": "Bahía bellas, Los Alcazares, Cartagena",
        "postalCode": "30368",
        "location": [37.7175, -0.8598],
        "unitCode": "MTR",
        "source": "https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06B01L03",
        "section": 1,
        "nextSection": None,
        "previousSection": None,
    }
}

#### Creación de la Red Acuatica que representa la rmbla del Albujon

In [28]:
import requests
id_water_network = "urn:ngsi-ld:WaterNetwork:000"

crear_water_network(
    id = str(0).zfill(3), 
    isComposedOf = [], 
    description = "Una de las ramblas mas importantes de la Región compuesta a su vez por un conjunto de otras ramblas", 
    name = "Rambla del Albujón", 
)

WaterNetwork
201


#### Lectura de los csv/parquet para juntar los datos

In [29]:
import os
import pandas as pd
import glob
import pyarrow.parquet as pq
import numpy as np
# Especifica la ruta de la carpeta principal
carpeta = "/home/thinking/raw/SAIHdatasActualizados/Ramblas/"

# Utiliza os.listdir() para obtener una lista de elementos en la carpeta
elementos_en_carpeta = os.listdir(carpeta)

# Filtra solo las carpetas en la lista de elementos
carpetas = [elemento for elemento in elementos_en_carpeta if os.path.isdir(os.path.join(carpeta, elemento))]

# DataFrame vacío para almacenar los datos
dfs = {}


# Imprime la lista de carpetas
for carpeta_actual in carpetas:
    df_total = pd.DataFrame()
    archivos_parquet = glob.glob(os.path.join(carpeta+carpeta_actual, "*.parquet"))
    for archivo in archivos_parquet:

        df_temporal = pd.read_parquet(archivo)
        tabla = pq.ParquetFile(archivo)

        #df_temporal.to_csv(archivo.split(".")[0]+"from_parquet.csv", index=False)
        
        if(len(df_total) == 0):
            df_total = df_temporal
        else:
            #Hacer un merge de df_total y df_temporal por la columna "Date"
            df_total = pd.merge(df_total, df_temporal, on="Date", how="outer")
    dfs[carpeta_actual] = df_total

for key, df in dfs.items():
    # Convierte la columna "Date" a tipo datetime
    df["Date"] = pd.to_datetime(df["Date"])
    # Formatea la columna "Date" en el nuevo formato
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

for key, df in dfs.items():
    numeric_cols = df.columns.copy()
    numeric_cols = numeric_cols.drop("Date")
    df.replace('-', float('nan'), inplace=True)
    df.replace("null", float('nan'), inplace=True)
    df[numeric_cols] = df[numeric_cols].astype(float)
    df["Date"] = pd.to_datetime(df["Date"])
    # Formatea la columna "Date" en el nuevo formato
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")

#### Codigo para crear las ramblas y los dispositivos 

In [30]:
import math
import time

#num_devices = 31
#counter_device_measurements = 1300

# Mismos Diccionarios que en el apartado de Boyas
maper_property_sensor = { "Volumen":"VolumeSensor","Presion":"PressureSensor","Temperatura": "TemperatureSensor", "Presion Atmosferica":"AtmosphericPressureSensor","Direccion del Viento":"WindDirectionSensor","Velocidad del Viento":"WindSpeedSensor" ,"Nivel": "WaterLevelSensor", "Pluviometro":"PrecipitationSensor","Caudal":"WaterFlowSensor", "Humedad":"HumiditySensor","Conductividad":"ConductivitySensor",
}

maper_property_controlled = { "Volumen":"volume","Presion":"pressure","Presion Atmosferica":"atmosphericPressure","Direccion del Viento":"windDirection","Velocidad del Viento":"windSpeed","Temperatura": "temperature", "Nivel": "waterLevel", "Pluviometro":"precipitation","Caudal":"waterFlow", "Humedad":"humidity","Conductividad":"conductivity"
}

maper_property_unitcode = { "Volumen":"m3","Presion":"bar", "Presion Atmosferica":"mbar", "Direccion del Viento":"°","Velocidad del Viento":"m/s","Temperatura": "CEL", "Conductividad": "microseconds/cm", "Nivel": "m", "Pluviometro":"mm","Caudal":"m3/s", "Humedad":"%"
}


# Diccionario que asocia el nombre de la carpeta a la propiedad (CREO Q NO SE USA)
maper_property_filename = {
    "Volumen Deposito":"B01-Volumen Depositofrom_parquet.csv","Presion Tuberia Salida Impulsion":"D03-Presion Tuberia Salida Impulsionfrom_parquet.csv","Presion Atmosferica":"D03-Presion Atmosfericafrom_parquet.csv",
    "Direccion del Viento":"D04-Direccion del Vientofrom_parquet.csv","Velocidad del Viento":"D09-Velocidad del Vientofrom_parquet.csv","Temperatura": "D05-Temperaturafrom_parquet.csv", 
    "Nivel": "U12-Nivelfrom_parquet.csv", "Pluviometro":"P01-Pluviometrofrom_parquet.csv","Caudal":"Q01-Caudalfrom_parquet.csv", "Humedad":"D06-Humedadfrom_parquet.csv",
    "Conductividad Aspiracion":"E03-Conductividad Aspiracionfrom_parquet.csv", "Nivel sobre Vertedero":"L01-Nivel sobre Vertederofrom_parquet.csv",
    "Nivel Deposito":"L02-Nivel Depositofrom_parquet.csv","Nivel Captacion": "L03-Nivel Captacionfrom_parquet.csv", "Caudal Salida al Mar": "Q01-Caudal Salida al Marfrom_parquet.csv",
    "Caudal Salida a RmblAlbujon":"Q02-Caudal Salida a RmblAlbujonfrom_parquet.csv", "Caudal Salida ElvAlbujon a ElvNarejos":"Q03-Caudal Salida ElvAlbujon a ElvNarejosfrom_parquet.csv"
}


# Diccionario para asociad device measurement con las entidades
link_rambla_sensores = {}
## Diccionario que asocia rambla con la carpeta
rambla_entity = {}
num_ramblas = 0

# En este diccionario se indican las ramblas que pertenece a la rambla del Albujón
# Se rellenara con el id de la entidad asociada a cada una de las ramblas
ramblas_albujon = {
    "06A06": "",
    "06A05": "",
    "06A04": "",
    "06A03": "",
    "06A02": "",
    "06A18": "",
    "06A01": ""
}

# Para cada rambla
for carpeta, valorCarpeta in dfs.items():
    print(carpeta)
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))

    # Obtenemos id de la rambla
    id_rambla = carpeta.split("-")[0]
    id_entity = maper_rambla_id[id_rambla]

    # Añadimos el id de la rambla a la lista de ramblas de la rambla del Albujón
    if id_rambla in ramblas_albujon:
        ramblas_albujon[id_rambla] = id_entity

    # obtenemos numero entidad y dispositivo
    number_entity = id_entity.split(":")[-1]
    number_device = str(num_devices).zfill(3)

    rambla_entity[carpeta] = id_entity
    
    if (id_rambla not in diccionario_info_ramblas):
        continue

    controlled_properties = []

    # Se añaden las propiedades controladas, hay que tener en cuenta que hay algunas repetidas
    # como Nivel deposito y Nivel capacitación, en ese caso almacenamos solo "Nivel"
    for valor in sensores:
        if valor != "Date":
            if valor in maper_property_controlled:
                controlled_properties.append(maper_property_controlled[valor])
            else:
                controlled_properties.append(valor.split(" ")[0])

    # Creamos el dispositivo
    crear_device(
        id=number_device,
        alternateName = "Multiple sensors for Ravine "+ id_rambla,
        controlled_entity = id_entity,
        controlled_properties = controlled_properties,
        description = "Device placed in Ravine "+ id_rambla+" that flow into Mar Menor",
        postalCode = diccionario_info_ramblas[id_rambla]["postalCode"],
        locality = diccionario_info_ramblas[id_rambla]["locality"],
        location = diccionario_info_ramblas[id_rambla]["location"],
        name = "Device " + number_device+" found in Piezometric Point "+ id_rambla,
        source = diccionario_info_ramblas[id_rambla]["source"],
        dateLastValueReported=valorCarpeta["Date"].iloc[len(valorCarpeta)-1]
    )

    # Para cada fila del csv
    for index, row in valorCarpeta.iterrows():
        # Si es la primera fila creamos las entidades
        if index == 0:
            for contador,valorSensor in enumerate(sensores):
                original = valorSensor
                if valorSensor == "Date":
                    continue

                id_device_measurement = f"urn:ngsi-ld:DeviceMeasurement:{str(counter_device_measurements).zfill(3)}"

                # Si el nombre del sensor tiene más de una palabra y no esta en el diccionario de propiedades
                # la segunda se considera el lugar de la medición
                measurementPlace = None
                if len(valorSensor.split(" "))>0 and valorSensor not in maper_property_controlled:
                    measurementPlace = " ".join(valorSensor.split(" ")[1:])
                    valorSensor = valorSensor.split(" ")[0]
                
                crear_device_measurement(
                    id=str(counter_device_measurements).zfill(3),
                    deviceType=maper_property_sensor[valorSensor],
                    value=row[original],
                    controlledProperty=maper_property_controlled[valorSensor],
                    observedAt=row["Date"],
                    depth=None,
                    target=measurementPlace,
                    refDevice=number_device,
                    unitCode=maper_property_unitcode[valorSensor],
                    name="DeviceMeasurement "+ str(counter_device_measurements).zfill(3),
                    source=diccionario_info_ramblas[id_rambla]["source"],
                    dateLastValueReported=row["Date"]
                )

                counter_device_measurements = counter_device_measurements + 1
                
                # Asociamos el id del dispositivo de medicion con el id de la entidad y el nombre de la propiedad controlada
                if id_entity not in link_rambla_sensores:
                    link_rambla_sensores[id_entity] = {}
                if valorSensor not in link_rambla_sensores[id_entity]:
                    link_rambla_sensores[id_entity][valorSensor] = {}

                link_rambla_sensores[id_entity][valorSensor][measurementPlace] = id_device_measurement
    
            entities_id[id_entity] = diccionario_info_ramblas[id_rambla]["location"]
            
            # Creamos la rambla
            crear_rambla(
                id=number_entity,
                dateLastValueReported=row["Date"],
                description=diccionario_info_ramblas[id_rambla]["description"],
                locality=diccionario_info_ramblas[id_rambla]["locality"],
                postalCode=diccionario_info_ramblas[id_rambla]["postalCode"],
                source=diccionario_info_ramblas[id_rambla]["source"],
                location=diccionario_info_ramblas[id_rambla]["location"],
                section=diccionario_info_ramblas[id_rambla]["section"],
                nextSection=diccionario_info_ramblas[id_rambla]["nextSection"],
                previousSection=diccionario_info_ramblas[id_rambla]["previousSection"],
                name="Ravine " + id_rambla,
                isPartOf=id_water_network if id_rambla in ramblas_albujon else None
            )

            num_ramblas += 1

        # Si el indice es 1 entonces haremos la modificación de las entidades
        elif index == 1:
            counters_valorSensor= {}
            for contador,valorSensor in enumerate(sensores):
                if valorSensor == "Date":
                    continue
                original = valorSensor
                
                # Mismo proceso que el de arriba
                measurementPlace = None
                if len(valorSensor.split(" "))>0 and valorSensor not in maper_property_controlled:
                    measurementPlace = " ".join(valorSensor.split(" ")[1:])
                    valorSensor = valorSensor.split(" ")[0]

                if valorSensor not in counters_valorSensor:
                    counters_valorSensor[valorSensor] = 0

                # Recuperamos el ide del dispositivo de medicion y modificamos la entidad
                id_device_measurement = link_rambla_sensores[id_entity][valorSensor][measurementPlace]
                counters_valorSensor[valorSensor] += 1

                modify_entity(
                    value=row[original],
                    observedAt=row["Date"],
                    unitcode=maper_property_unitcode[valorSensor],
                    url="http://localhost:1026/ngsi-ld/v1/entities/"+id_device_measurement+"/attrs",
                )
                time.sleep(0.2)
        else:
            break
    num_devices = num_devices+1

with open('./variables_entidades/rambla_entity.pkl', 'wb') as f:
    pickle.dump(rambla_entity, f)
with open('./variables_entidades/link_rambla_sensores.pkl', 'wb') as f:
    pickle.dump(link_rambla_sensores, f)

06A02-Pozo Estrecho
Device
{'id': 'urn:ngsi-ld:Device:031', 'alternateName': {'type': 'Property', 'value': 'Multiple sensors for Ravine 06A02'}, 'areaServed': {'type': 'Property', 'value': 'Mar Menor'}, 'type': 'Device', 'controlledAsset': {'type': 'Relationship', 'object': ['urn:ngsi-ld:Ravine:002']}, 'controlledProperty': {'type': 'Property', 'value': ['precipitation', 'waterLevel', 'temperature', 'waterFlow']}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}, 'description': {'type': 'Property', 'value': 'Device placed in Ravine 06A02 that flow into Mar Menor'}, 'deviceCategory': {'type': 'Property', 'value': ['sensor']}, 'address': {'type': 'Property', 'value': {'addressCountry': 'ES', 'addressRegion': 'Murcia', 'addressLocality': 'Pozo Estrecho, Cartagena', 'postalCode': '30594'}, 'verified': {'type': 'Property', 'value': True}}, 'source': {'type': 'Property', 'value': 'https://saihweb.chsegura.es/apps/iVisor/visor_variable.php?punto=06A02U12'}, 'loca

#### Subimos los historicos al Timescale

In [31]:
import psycopg2
from datetime import datetime
import uuid
# Conectarse a la base de datos
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="orion",
    user="orion",
    password="orion"
)

cursor = conn.cursor()
datos_a_insertar = []

maper_property_controlled = { "Volumen":"volume","Presion":"pressure","Presion Atmosferica":"atmosphericPressure","Direccion del Viento":"windDirection","Velocidad del Viento":"windSpeed","Temperatura": "temperature", "Nivel": "waterLevel", "Pluviometro":"precipitation","Caudal":"waterFlow", "Humedad":"humidity","Conductividad":"conductivity"
}
maper_property_unitcode = { "Volumen":"m3","Presion":"bar", "Presion Atmosferica":"mbar", "Direccion del Viento":"°","Velocidad del Viento":"m/s","Temperatura": "CEL", "Conductividad": "microseconds/cm", "Nivel": "m", "Pluviometro":"mm","Caudal":"m3/s", "Humedad":"%"
}
for carpeta, valorCarpeta in dfs.items():
    id_entity = rambla_entity[carpeta]
    print(carpeta)
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    for index, row in valorCarpeta.iterrows():
        if index == len(valorCarpeta)-1 or index == 0 or index == 1:
            continue
        counters_valorSensor= {}
        for contador,valorSensor in enumerate(sensores):
            if valorSensor == "Date":
                continue
            original = valorSensor
        
            
            measurementPlace = None
            if len(valorSensor.split(" "))>0 and valorSensor not in maper_property_controlled:
                measurementPlace = " ".join(valorSensor.split(" ")[1:])
                valorSensor = valorSensor.split(" ")[0]

            id_device_measurement = link_rambla_sensores[id_entity][valorSensor][measurementPlace]

            fecha_hora = datetime.strptime(row["Date"], "%Y-%m-%dT%H:%M:%SZ")

            value = row[original]
            if math.isnan(row[original]):
                    value = -99

            datos_a_insertar.append(
                (f'urn:ngsi-ld:attributes3:{uuid.uuid4()}', 'https://smartdatamodels.org/dataModel.DeviceMeasurement/numValue', 'Replace', id_device_measurement, fecha_hora, True, maper_property_unitcode[valorSensor], 'None', 'Number', None, None, value, None, None, None, None, None, None, None, None, datetime.now())
            )
            
            if len(datos_a_insertar) > 100000:
                consulta_insercion = f"INSERT INTO attributes VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                print(f"Insertando {len(datos_a_insertar)} filas en la tabla attributes")
                try:
                    # Ejecutar la consulta para insertar los datos
                    cursor.executemany(consulta_insercion, datos_a_insertar)
                    conn.commit()
                    datos_a_insertar = []
                    print(f"Se han insertado {cursor.rowcount} filas en la tabla attributes")
                except (Exception, psycopg2.DatabaseError) as error:
                    conn.rollback()
                # Confirmar la transacción
                    print(f"Error: {error}")

if len(datos_a_insertar) > 0:
    consulta_insercion = f"INSERT INTO attributes VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    print(f"Insertando {len(datos_a_insertar)} filas en la tabla attributes")
    try:
        # Ejecutar la consulta para insertar los datos
        cursor.executemany(consulta_insercion, datos_a_insertar)
        conn.commit()
        datos_a_insertar = []
        print(f"Se han insertado {cursor.rowcount} filas en la tabla attributes")
    except (Exception, psycopg2.DatabaseError) as error:
        conn.rollback()
    # Confirmar la transacción
        print(f"Error: {error}")


# Cerrar el cursor y la conexión
cursor.close()
conn.close()

06A02-Pozo Estrecho
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en la tabla attributes
Insertando 100001 filas en la tabla attributes
Se han insertado 100001 filas en 

#### Actualizamos el ORION con los ultimos valores

In [32]:
for carpeta, valorCarpeta in dfs.items():
    ultimos_valores = valorCarpeta.iloc[len(valorCarpeta)-1]
    print(carpeta)
    sensores = list(set( val.split("_")[0] for val in valorCarpeta.columns))
    id_entity = rambla_entity[carpeta]
    for contador,valorSensor in enumerate(sensores):
        if valorSensor == "Date":
            continue
        original = valorSensor
        counters_valorSensor= {}
        
        measurementPlace = None
        if len(valorSensor.split(" "))>0 and valorSensor not in maper_property_controlled:
            measurementPlace = " ".join(valorSensor.split(" ")[1:])
            valorSensor = valorSensor.split(" ")[0]

        if valorSensor not in counters_valorSensor:
            counters_valorSensor[valorSensor] = 0

        id_device_measurement = link_rambla_sensores[id_entity][valorSensor][measurementPlace]
        counters_valorSensor[valorSensor] += 1
        
        modify_entity(
            value=ultimos_valores[original],
            observedAt=ultimos_valores["Date"],
            unitcode=maper_property_unitcode[valorSensor],
            url="http://localhost:1026/ngsi-ld/v1/entities/"+id_device_measurement+"/attrs",
        )

06A02-Pozo Estrecho
modify_content
{'numValue': {'type': 'Property', 'value': 0.0, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'mm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 0.0, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'm'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 20.1, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'CEL'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
modify_content
{'numValue': {'type': 'Property', 'value': 0.0, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'm3/s'}, 'dateLastValueReported': {'type': 'Property', 'value': '2024-06-09T23:55:00Z'}}
204
06P04-La Murta
modify_content
{'numValue': {'type': 'Property', 'value': 0.0, 'observedAt': '2024-06-09T23:55:00Z', 'unitCode': 'mm'}, 'dateLastValueReported': {

#### Modificamos la Red Acuatica para añadirle el ID de las Ramblas

In [33]:

entidades_albujon = ramblas_albujon.values()

value_to_patch = []
for entidad in entidades_albujon:
    patch_content = {
        "type": "Relationship",
        "object": entidad,
    }
    value_to_patch.append(patch_content)
    

modify_entity(
    value=value_to_patch,
    observedAt=None,
    unitcode=None,
    url="http://localhost:1026/ngsi-ld/v1/entities/"+id_water_network+"/attrs",
    isComposedOf=True
)

modify_content
{'isComposedOf': [{'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:006'}, {'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:005'}, {'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:004'}, {'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:003'}, {'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:002'}, {'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:007'}, {'type': 'Relationship', 'object': 'urn:ngsi-ld:Ravine:001'}]}
204


#### Añadimos a cada entidad sus entidades cercanas 

In [34]:
entities_id

{'urn:ngsi-ld:Buoy:000': [37.76, -0.78],
 'urn:ngsi-ld:Buoy:001': [37.68, -0.78],
 'urn:ngsi-ld:Buoy:002': [37.65, -0.72],
 'urn:ngsi-ld:Buoy:003': [37.65, -0.78],
 'urn:ngsi-ld:Buoy:004': [37.66, -0.8],
 'urn:ngsi-ld:Buoy:005': [37.74, -0.74],
 'urn:ngsi-ld:Buoy:006': [37.81, -0.78],
 'urn:ngsi-ld:Buoy:007': [37.71, -0.83],
 'urn:ngsi-ld:Buoy:008': [37.69, -0.81],
 'urn:ngsi-ld:Buoy:009': [37.71, -0.77],
 'urn:ngsi-ld:Buoy:010': [37.74, -0.72],
 'urn:ngsi-ld:Buoy:011': [37.76, -0.8],
 'urn:ngsi-ld:SoundingPlace:000': [37.7058, -0.8633],
 'urn:ngsi-ld:SoundingPlace:001': [37.8181, -0.8286],
 'urn:ngsi-ld:SoundingPlace:002': [37.7755, -0.8514],
 'urn:ngsi-ld:SoundingPlace:003': [37.7543, -0.8588],
 'urn:ngsi-ld:SoundingPlace:004': [37.6841, -0.8692],
 'urn:ngsi-ld:SoundingPlace:005': [37.7104, -0.8879],
 'urn:ngsi-ld:SoundingPlace:006': [37.7511, -0.8828],
 'urn:ngsi-ld:SoundingPlace:007': [37.654261, -0.848734],
 'urn:ngsi-ld:SoundingPlace:008': [37.7696, -0.8385],
 'urn:ngsi-ld:Soundi

In [35]:
for entity_id, location in entities_id.items():
    close_entities = search_close_entities(location[0], location[1],entity_id)
    print(entity_id)
    create_attribute(
        multipleValue=[{"type":"Relationship","object": close_entity} for close_entity in close_entities],
        value=None,
        observedAt=None,
        url="http://localhost:1026/ngsi-ld/v1/entities/"+entity_id+"/attrs",
        newAttributeName="closeMeasurements"
    )

urn:ngsi-ld:Buoy:000
urn:ngsi-ld:Buoy:001
urn:ngsi-ld:Buoy:002
urn:ngsi-ld:Buoy:003
urn:ngsi-ld:Buoy:004
urn:ngsi-ld:Buoy:005
urn:ngsi-ld:Buoy:006
urn:ngsi-ld:Buoy:007
urn:ngsi-ld:Buoy:008
urn:ngsi-ld:Buoy:009
urn:ngsi-ld:Buoy:010
urn:ngsi-ld:Buoy:011
urn:ngsi-ld:SoundingPlace:000
urn:ngsi-ld:SoundingPlace:001
urn:ngsi-ld:SoundingPlace:002
urn:ngsi-ld:SoundingPlace:003
urn:ngsi-ld:SoundingPlace:004
urn:ngsi-ld:SoundingPlace:005
urn:ngsi-ld:SoundingPlace:006
urn:ngsi-ld:SoundingPlace:007
urn:ngsi-ld:SoundingPlace:008
urn:ngsi-ld:SoundingPlace:009
urn:ngsi-ld:SoundingPlace:010
urn:ngsi-ld:SoundingPlace:011
urn:ngsi-ld:SoundingPlace:012
urn:ngsi-ld:SoundingPlace:013
urn:ngsi-ld:SoundingPlace:014
urn:ngsi-ld:SoundingPlace:015
urn:ngsi-ld:SoundingPlace:016
urn:ngsi-ld:SoundingPlace:017
urn:ngsi-ld:SoundingPlace:018
urn:ngsi-ld:Ravine:002
urn:ngsi-ld:Ravine:010
urn:ngsi-ld:Ravine:009
urn:ngsi-ld:Ravine:005
urn:ngsi-ld:Ravine:011
urn:ngsi-ld:Ravine:008
urn:ngsi-ld:Ravine:004
urn:ngsi-ld:Ravin